In [1]:
import pandas as pd
from datetime import datetime

In [2]:
result_cols = ["Bib", "Name", "Run1", "Run2"]
#result_cols_keep = ["Bib", "Name", "Run1", "Run2"]
startList_cols_keep = ["Bib", "Name", "Team", "Club"]

In [3]:
results1 = pd.read_csv('data/140124FnGResultsBoth.csv', header=None, names=result_cols)
results2 = pd.read_csv('data/210124FnGResultsBoth.csv', header=None, names=result_cols)
startList = pd.read_csv('data/24FnGStartList.csv')[startList_cols_keep]

In [4]:
startList.columns = ["Bib", "Name", "Tier", "Club"]

In [5]:
combined1 = startList.merge(results1, how="left", on=["Bib", "Name"])
combined2 = startList.merge(results2, how="left", on=["Bib", "Name"])

In [6]:
def calculate_points_corrected(df):
    df.replace({'DNF': 9998,"DSQ": 9998, "DNS": 9999, pd.NA: 9999}, inplace=True)
    df['Run1'] = pd.to_numeric(df['Run1']) # errors='coerce')
    df['Run2'] = pd.to_numeric(df['Run2']) # errors='coerce')
    df['Best Time'] = df[['Run1', 'Run2']].min(axis=1)
    # Initialize a column for points
    df['Points'] = 0
    
    # Process each tier
    for tier in df['Tier'].unique():
        # Filter the tier
        tier_df = df[df['Tier'] == tier]
    
        # Sort by best time
        tier_sorted = tier_df.sort_values(by='Best Time')
    
        # Assign points based on the number of racers in the tier
        num_racers = len(tier_sorted)
        tier_sorted['Points'] = range(8, 8-num_racers, -1)
    
        # Set absent racer points to zero:
        tier_sorted.loc[tier_sorted['Best Time'] == 9999, 'Points'] = 0
        
        # Handle ties for DNF (split points for racers with 9998 as their best time)
        dnf_racers = tier_sorted[tier_sorted['Best Time'] == 9998]
        if not dnf_racers.empty:
            dnf_points = dnf_racers['Points'].sum() / len(dnf_racers)
            tier_sorted.loc[tier_sorted['Best Time'] == 9998, 'Points'] = dnf_points
    
        # Update the main dataframe
        df.loc[df['Tier'] == tier, 'Points'] = tier_sorted['Points']

    # Calculate team points
    teams = df['Club'].unique()
    team_dfs = []
    team_points = {}
    for team in teams:
        team_df = df[df['Club'] == team]
        # If a team is missing a racer in a tier, give them the average points of that tier
        for tier in df['Tier'].unique():
            if tier not in team_df['Tier'].values:
                #avg_points = df[df['Tier'] == tier]['Points'].mean()
                #team_df.loc[len(team_df)] = [pd.NA, pd.NA, tier, team, pd.NA, pd.NA, pd.NA, avg_points]
                team_df.loc[-1] = [pd.NA, pd.NA, tier, team, pd.NA, pd.NA, pd.NA, 4.5] #, avg_points]
                team_df.reset_index(drop=True, inplace=True)
        team_points[team] = team_df['Points'].sum()
        team_dfs.append(team_df)
    
    df_out = pd.concat(team_dfs)
    
    return team_points, df_out
    

In [7]:
team_points1, _ = calculate_points_corrected(combined1)

/var/folders/xl/qrck89yn5kq2bhd8vf0657700000gn/T/ipykernel_97640/3388888520.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  team_df.loc[-1] = [pd.NA, pd.NA, tier, team, pd.NA, pd.NA, pd.NA, 4.5] #, avg_points]
/var/folders/xl/qrck89yn5kq2bhd8vf0657700000gn/T/ipykernel_97640/3388888520.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_df.loc[-1] = [pd.NA, pd.NA, tier, team, pd.NA, pd.NA, pd.NA, 4.5] #, avg_points]
/var/folders/xl/qrck89yn5kq2bhd8vf0657700000gn/T/ipykernel_97640/3388888520.py:44: FutureWarning: The behavior of DataFrame concatena

In [8]:
team_points2, df_out = calculate_points_corrected(combined2)

/var/folders/xl/qrck89yn5kq2bhd8vf0657700000gn/T/ipykernel_97640/3388888520.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  team_df.loc[-1] = [pd.NA, pd.NA, tier, team, pd.NA, pd.NA, pd.NA, 4.5] #, avg_points]
/var/folders/xl/qrck89yn5kq2bhd8vf0657700000gn/T/ipykernel_97640/3388888520.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  team_df.loc[-1] = [pd.NA, pd.NA, tier, team, pd.NA, pd.NA, pd.NA, 4.5] #, avg_points]
/var/folders/xl/qrck89yn5kq2bhd8vf0657700000gn/T/ipykernel_97640/3388888520.py:44: FutureWarning: The behavior of DataFrame concatena

In [16]:
for i in range (13, 0, -1):
    #print(df_out[df_out.Tier == i].sort_values(["Tier", 'Points'], ascending=False).shape)
    assert df_out[df_out.Tier == i].sort_values(["Tier", 'Points'], ascending=False).shape == (8,8)

In [9]:
team_points1

{'Mitch Perreault': 52,
 'Will Carter': 73,
 'Stephanie Coward': 55.0,
 'Mike McTaggart': 64,
 'Don French ': 67,
 'Joanna Perreault': 50.5,
 'Adam Grossman': 68,
 'Graham Ramshaw ': 41.5}

In [10]:
for team, new_points in team_points2.items():
    team_points1[team] += new_points

In [11]:
team_points1

{'Mitch Perreault': 97,
 'Will Carter': 145,
 'Stephanie Coward': 107.0,
 'Mike McTaggart': 140,
 'Don French ': 132,
 'Joanna Perreault': 111.0,
 'Adam Grossman': 119,
 'Graham Ramshaw ': 89.0}

In [12]:
df_out.to_csv("210124_scores_groupbyClub.csv")

In [13]:
df_out.sort_values(["Tier", 'Points'], ascending=False).to_csv("210124_scores_groupbyTier.csv")

In [14]:
df_out[df_out.Tier == 13].sort_values(["Tier", 'Points'], ascending=False).head(10)

,Bib,Name,Tier,Club,Run1,Run2,Best Time,Points
96,120,Hannah Kilmer Choi,13,Will Carter,38.07,9999.00,38.07,8.0
10,119,Jenna Livingston,13,Stephanie Coward,42.45,41.35,41.35,7.0
11,121,Graham Ramshaw,13,Graham Ramshaw,44.74,42.34,42.34,6.0
98,122,Kevin Kilmer Choi,13,Adam Grossman,42.47,9999.00,42.47,5.0
12,<NA>,NaN,13,Joanna Perreault,NaN,NaN,NaN,4.5
93,116,Melanie Leistner Lavoie,13,Mike McTaggart,42.87,42.92,42.87,4.0
99,123,Bernard Oegema,13,Don French,46.00,44.35,44.35,3.0
94,117,Kyla Taylor,13,Mitch Perreault,52.10,50.80,50.80,2.0


(8, 8)
(8, 8)
(8, 8)
(8, 8)
(8, 8)
(8, 8)
(8, 8)
(8, 8)
(8, 8)
(8, 8)
(8, 8)
(8, 8)
(8, 8)
